In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import euclidean_distances


In [4]:
# Load dataset
df = pd.read_csv("electricvehicles.csv")

# Drop unused column
df = df.drop(columns=["Segment"])

# Clean and preprocess
df = df.dropna(subset=[
    "AccelSec", "TopSpeed_KmH", "Range_Km", "Efficiency_WhKm",
    "FastCharge_KmH", "RapidCharge", "Seats", "PriceEuro"
])

# Encode categorical values
powertrain_map = {"AWD": 1.0, "RWD": 0.75, "FWD": 0.5}
df["PowerTrainScore"] = df["PowerTrain"].map(powertrain_map).fillna(0.5)
df["RapidChargeScore"] = df["RapidCharge"].apply(lambda x: 1 if str(x).lower() == "yes" else 0)

# Select numerical columns for scoring (excluding PriceEuro from score)
score_columns = [
    "AccelSec", "TopSpeed_KmH", "Range_Km", "Efficiency_WhKm",
    "FastCharge_KmH", "Seats", "RapidChargeScore", "PowerTrainScore"
]

# Normalize numerical features
scaler = MinMaxScaler()
norm_features = scaler.fit_transform(df[score_columns])

# Define ideal (best possible) vehicle: min(AccelSec, Efficiency), max(all others)
ideal_point = []
for col in score_columns:
    if col in ["AccelSec", "Efficiency_WhKm"]:
        ideal_point.append(0.0)  # min is best after normalization
    else:
        ideal_point.append(1.0)  # max is best after normalization

# Compute Euclidean distance to ideal vehicle
distances = euclidean_distances(norm_features, [ideal_point]).flatten()
df["DistanceToIdeal"] = distances

# Compute value score as inverse of distance per euro
# Add small constant to avoid division by zero
epsilon = 1e-6
df["ValueScore"] = 1 / (df["DistanceToIdeal"] + epsilon) / df["PriceEuro"]

# Sort and show top result
top_vehicle = df.sort_values("ValueScore", ascending=False).iloc[0]

# Output top vehicle details
print("Top Value-per-Euro Vehicle (Euclidean method):")
print(top_vehicle[["Brand", "Model", "ValueScore"]])


Top Value-per-Euro Vehicle (Euclidean method):
Brand                 SEAT
Model         Mii Electric
ValueScore        0.000019
Name: 44, dtype: object


In [5]:
df.sort_values("ValueScore", ascending=False)

,Brand,Model,AccelSec,TopSpeed_KmH,Range_Km,Efficiency_WhKm,FastCharge_KmH,RapidCharge,PowerTrain,PlugType,BodyStyle,Seats,PriceEuro,PowerTrainScore,RapidChargeScore,DistanceToIdeal,ValueScore
44,SEAT,Mii Electric,12.3,130,195,166,170.0,Yes,FWD,Type 2 CCS,Hatchback,4,20129,0.5,1,2.599918,0.000019
17,Volkswagen,e-Up!,11.9,130,195,166,170.0,Yes,FWD,Type 2 CCS,Hatchback,4,21421,0.5,1,2.589031,0.000018
74,Sono,Sion,9.0,140,225,156,270.0,Yes,FWD,Type 2 CCS,Hatchback,5,25500,0.5,1,2.320500,0.000017
43,Skoda,CITIGOe iV,12.3,130,195,166,170.0,Yes,FWD,Type 2 CCS,Hatchback,4,24534,0.5,1,2.599918,0.000016
20,Opel,Corsa-e,8.1,150,275,164,420.0,Yes,FWD,Type 2 CCS,Hatchback,5,29146,0.5,1,2.203678,0.000016
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48,Lightyear,One,10.0,150,575,104,540.0,Yes,AWD,Type 2 CCS,Liftback,5,149000,1.0,1,1.794299,0.000004
79,Porsche,Taycan Turbo,3.2,260,390,215,810.0,Yes,AWD,Type 2 CCS,Sedan,4,148301,1.0,1,1.807547,0.000004
72,Porsche,Taycan Cross Turismo,3.5,250,385,217,770.0,Yes,AWD,Type 2 CCS,Station,4,150000,1.0,1,1.831717,0.000004
51,Tesla,Roadster,2.1,410,970,206,920.0,Yes,AWD,Type 2 CCS,Cabrio,4,215000,1.0,1,1.537839,0.000003
